In [17]:
import cv2
import numpy as np
import os
    
# Bulk testing to find the metrics for the different values of match-threshold and distance-threshold

# Enhance image quality using CLAHE (Contrast Limited Adaptive Histogram Equalization)
def enhance_image(image):
    """Enhance image quality."""
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(image)
    return enhanced_image

# Load images from a folder and apply enhancement
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), 0)  # Load as grayscale
        if img is not None:
            img = enhance_image(img)  # Enhance the image
            images.append((filename, img))
    return images

# Check similarity between template and image using ORB feature detection and matching
def is_similar(template, image, match_threshold=15, distance_threshold=30):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(template, None)
    kp2, des2 = orb.detectAndCompute(image, None)

    if des1 is None or des2 is None:  # If no descriptors are found
        return False

    # Match descriptors using BFMatcher with Hamming distance and crossCheck
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)  # Sort by distance

    # Filter "good matches" based on the distance threshold
    good_matches = [m for m in matches if m.distance < distance_threshold]
    return len(good_matches) > match_threshold  # Return True if enough good matches

# Test the model by evaluating performance on both correct and incorrect image sets
def test_model(template_path, correct_folder, incorrect_folder, match_threshold=11, distance_threshold=47):
   
    # Print the current match and distance thresholds
    print(f"Testing with match_threshold = {match_threshold} and distance_threshold = {distance_threshold}")

    template = cv2.imread(template_path, 0)  # Load template as grayscale
    if template is None:
        raise FileNotFoundError(f"Template image '{template_path}' not found.")

    template = enhance_image(template)  # Enhance the template image

    # Load correct and incorrect images
    correct_images = load_images_from_folder(correct_folder)
    incorrect_images = load_images_from_folder(incorrect_folder)

    # Initialize performance metrics
    TP, TN, FP, FN = 0, 0, 0, 0

    # Evaluate correct images
    for filename, img in correct_images:
        if is_similar(template, img, match_threshold, distance_threshold):
            TP += 1  # True Positive
        else:
            FN += 1  # False Negative

    # Evaluate incorrect images
    for filename, img in incorrect_images:
        if not is_similar(template, img, match_threshold, distance_threshold):
            TN += 1  # True Negative
        else:
            FP += 1  # False Positive

    # Calculate performance metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) * 100
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  # True Positive Rate (Recall)
    TNR = TN / (TN + FP) if (TN + FP) > 0 else 0  # True Negative Rate (Specificity)
    balanced_accuracy = (TPR + TNR) / 2  # Balanced Accuracy
    
    # Print performance results
    print(f"Correctly identified (True Positives): {TP}")
    print(f"Correctly rejected (True Negatives): {TN}")
    print(f"Incorrectly identified (False Positives): {FP}")
    print(f"Failed to identify (False Negatives): {FN}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1_score:.2f}")
    print(f"Model Accuracy: {accuracy:.2f}%")
    print(f"Balanced Accuracy: {balanced_accuracy:.2f}")

# Define file paths and folders
template_path = r"C:\Users\Khang Trung Nguyen\Amanda4\esp_captured\template_3.jpg"
correct_folder = r"C:\Users\Khang Trung Nguyen\Amanda4\augmented_esp_images"
incorrect_folder = r"data\incorrect_objects"

# Run the test
test_model(template_path, correct_folder, incorrect_folder)


Correctly identified (True Positives): 863
Correctly rejected (True Negatives): 911
Incorrectly identified (False Positives): 167
Failed to identify (False Negatives): 137
Precision: 0.84
Recall: 0.86
F1 Score: 0.85
Model Accuracy: 85.37%
Balanced Accuracy: 0.85


In [ ]:
#Testing one image with the best distance- and match-threshold

# Define a function to enhance the image quality using CLAHE
def enhance_image(image):
    """Enhance image quality by applying Contrast Limited Adaptive Histogram Equalization (CLAHE)."""
    # Apply CLAHE with clipLimit of 2.0 and tileGridSize of 8x8
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(image)  # Apply CLAHE to the image
    return enhanced_image

# Load the template (reference image) and the test image where you want to find the sticker
template = cv2.imread(r"C:\Users\Khang Trung Nguyen\Amanda4\esp_captured\template_3.jpg", 0)  # Load template in grayscale
image = cv2.imread(r"C:\Users\Khang Trung Nguyen\Amanda4\esp_captured\test_a1.jpg", 0)  # Load test image in grayscale

# Enhance both images using the enhance_image function
template = enhance_image(template)  # Enhance the template image
image = enhance_image(image)  # Enhance the test image

# Initialize ORB detector to find keypoints and descriptors
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors for both the template and the test image
kp1, des1 = orb.detectAndCompute(template, None)  # Detect keypoints and descriptors in template
kp2, des2 = orb.detectAndCompute(image, None)  # Detect keypoints and descriptors in test image

# Use BFMatcher with Hamming distance and cross-checking to match descriptors between template and image
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)  # Match the descriptors between the two images

# Sort matches based on distance, so the best matches come first
matches = sorted(matches, key=lambda x: x.distance)

# Log the total number of matches found between the two images
total_matches = len(matches)
print(f"Total Matches: {total_matches}")

# Filter "good matches" by setting a distance threshold for the match quality
distance_threshold = 47  # Threshold for considering matches as "good"
good_matches = [m for m in matches if m.distance < distance_threshold]  # Keep only the matches below the threshold

# Log the number of good matches
good_matches_count = len(good_matches)
print(f"Good Matches: {good_matches_count}")

# Draw the top 10 matches between the template and the test image
matched_image = cv2.drawMatches(template, kp1, image, kp2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matched image using Matplotlib
plt.imshow(matched_image)
plt.title('Detected Sticker')  # Set title for the display window
plt.show()  # Display the image with matched keypoints

# Determine whether the object in the test image is similar to the template object based on the number of good matches
match_threshold = 11  # Set a threshold for the number of good matches to consider the objects similar

if good_matches_count > match_threshold:
    print("The object in the input image is similar to the object in the template.")
else:
    print("The object in the input image is NOT similar to the object in the template.")
